In [13]:
import pandas as pd
import numpy as np
import datetime as dt
import os
from keras import regularizers
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense,Dropout,Activation, Lambda
from keras.layers import Conv1D, Conv2D, MaxPooling1D, Flatten
from keras.layers import MaxPooling1D, MaxPooling2D,Reshape, BatchNormalization
from keras.optimizers import Adam

In [ ]:
path = ""
lob = pd.read_csv(os.path.join(path, "lob.csv"), infer_datetime_format=True)

In [ ]:
lob.columns = ["date", "time" ] + ["bid_price{}".format(i) for i in range(1, 6)] + \
    ["ask_price{}".format(i) for i in range(1, 6)] + ["bid_size{}".format(i) for i in range(1, 6)] + \
    ["ask_size{}".format(i) for i in range(1, 6)]

In [ ]:
# time insensitive features
lob["ask_price_mean"] = lob["bid_price_mean"] = lob["ask_volume_mean"] = lob["bid_volume_mean"] = 0
for i in range(1, level + 1):
    lob["spread{}".format(i)] = lob["ask_price{}".format(i)] - lob["bid_price{}".format(i)]
    lob["mid_price{}".format(i)] = (lob["ask_price{}".format(i)] + lob["bid_price{}".format(i)]) / 2
    if i < level:
        lob["ask_price_diff_{}-{}".format(i + 1, i)] = lob["ask_price" + str(i + 1)] - lob["ask_price" + str(i)]
        lob["bid_price_diff_{}-{}".format(i + 1, i)] = lob["bid_price" + str(i + 1)] - lob["bid_price" + str(i)]
        lob["ask_volume_diff_{}-{}".format(i + 1, i)] = lob["ask_volume" + str(i + 1)] - lob["ask_volume" + str(i)]
        lob["bid_volume_diff_{}-{}".format(i + 1, i)] = lob["bid_volume" + str(i + 1)] - lob["bid_volume" + str(i)]
    lob["ask_price_mean"] += lob["ask_price{}".format(i)] / level
    lob["bid_price_mean"] += lob["bid_price{}".format(i)] / level
    lob["ask_volume_mean"] += lob["ask_volume{}".format(i)] / level
    lob["bid_volume_mean"] += lob["ask_volume{}".format(i)] / level

In [ ]:
# time sensitive features
# to do

In [ ]:
# label
lob["mid_price"] = lob["mid_price1"]
steps = [1, 2, 3, 5, 10]
for step in steps:
    lob["mid_price_change{}".format(step)] = lob["mid_price"].diff(period=step, axis=0)
    lob["label{}".format(step)] = 0 + 1 * (lob["mid_price_change{}".format(step)] > 0) + \
        - 1 * (lob["mid_price_change{}".format(step)] < 0)

In [15]:
configs = {}
configs['batch_size'] = 100
configs['sequence_length'] = 100
configs['epochs'] = 100
configs['learning_rate'] = .001
configs['learning_rate_decay'] = 0
configs['opt'] = Adam(lr=configs['learning_rate'], decay=configs['learning_rate_decay'])
configs['save_dir'] = 'saved_models'
configs['nflag'] = True

trainday [0, 1, 2, 3, 4, 5] valdays [6, 7] colx range(0, 40) coly 148
{'batch_size': 50, 'sequence_length': 100, 'epochs': 100, 'learning_rate': 0.001, 'learning_rate_decay': 0, 'opt': <keras.optimizers.Adam object at 0xb26b9c240>, 'save_dir': 'saved_models', 'nflag': True}


In [16]:
model = Sequential()
model.add(Conv1D(16, kernel_size=2, strides=1, activation='relu', padding='causal', input_shape=(100,40)))                
model.add(Conv1D(16, kernel_size=2, dilation_rate=2, activation='relu', padding='causal'))
model.add(Conv1D(16, kernel_size=2, dilation_rate=4, activation='relu', padding='causal'))
model.add(Conv1D(16, kernel_size=2, dilation_rate=8, activation='relu', padding='causal'))
model.add(Conv1D(16, kernel_size=2, dilation_rate=16, activation='relu', padding='causal'))
model.add(Flatten())        
model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.40))
model.add(Dense(3, activation='softmax'))
opt = Adam(lr=configs['learning_rate'], decay=configs['learning_rate_decay'])
model.compile(loss="categorical_crossentropy", optimizer=configs['opt'], metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_11 (Conv1D)           (None, 100, 16)           1296      
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 100, 16)           528       
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 100, 16)           528       
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 100, 16)           528       
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 100, 16)           528       
_________________________________________________________________
flatten_3 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 64)                102464    
__________

In [ ]:
history = model.fit_generator(generator=train_gen,
            validation_data=val_gen,
            steps_per_epoch=steps_per_epoch,
            validation_steps=validation_steps,
            epochs=epochs,
            verbose=2)